In [ ]:
# pyculiarity의 모듈을 가져온다.
!pip install pyculiarity

     |████████████████████████████████| 156 kB 4.1 MB/s 
  Created wheel for pyculiarity: filename=pyculiarity-0.0.7-py32-none-any.whl size=24797 sha256=a899ded768752010d9c1f6e734bb0a29586e59d1669f8549163e1b1029d266e5
  Stored in directory: /root/.cache/pip/wheels/0b/e5/88/c2411d890e635fb1492e641b25fd1e1060a3592c3acb6a0f8d
Successfully built pyculiarity


In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Proj_WT/DataSets/airpolution/interpolate_linear.csv', parse_dates = ['측정일자'])

In [ ]:
df.head()

,측정일자,동대문구,종로구,중구,성동구,금천구,송파구,관악구,강동구,서대문구,노원구,마포구,동작구,서초구,구로구,영등포구,강남구,강서구,성북구,중랑구,강북구,양천구,도봉구,은평구,용산구,광진구
0,2010-01-01,35.0,36.0,33.0,31.0,33.0,32.0,36.0,29.0,32.0,30.0,34.0,32.0,32.0,33.0,31.0,36.0,34.0,33.0,33.0,33.0,36.0,31.0,29.0,34.0,32.0
1,2010-01-02,50.0,53.0,48.0,51.0,56.0,46.0,59.0,45.0,53.0,47.0,54.0,57.0,55.0,56.0,53.0,56.0,53.0,42.0,49.0,47.0,62.0,44.0,47.0,51.0,45.0
2,2010-01-03,50.0,44.0,48.0,46.0,45.0,46.0,50.0,42.0,45.0,45.0,51.0,44.0,49.0,48.0,48.0,50.0,49.0,46.0,52.0,44.0,50.0,49.0,47.0,49.0,46.0
3,2010-01-04,38.0,43.0,37.0,38.0,38.0,34.0,39.0,32.0,37.0,37.0,42.0,39.0,37.0,38.0,37.0,38.0,37.0,34.0,40.0,37.0,43.0,33.0,35.0,38.0,34.0
4,2010-01-05,49.0,48.0,49.0,50.0,45.0,47.0,52.0,38.0,56.0,41.0,54.0,48.0,51.0,54.0,52.0,50.0,54.0,44.0,50.0,40.0,53.0,46.0,48.0,51.0,48.0


In [ ]:
df[['측정일자','동대문구']]

,측정일자,동대문구
0,2010-01-01,35.0
1,2010-01-02,50.0
2,2010-01-03,50.0
3,2010-01-04,38.0
4,2010-01-05,49.0
...,...,...
4013,2020-12-27,59.0
4014,2020-12-28,59.0
4015,2020-12-29,60.0
4016,2020-12-30,30.0


In [ ]:
# 각 구별로 이상치 탐지 후 결측치로 만들어 주기
import numpy as np
from pyculiarity import detect_ts

# max_anoms를 0.1,0.15, 0.2, 0.25, 0.3으로 바꿔가며 진행했다.
for i in df.columns[1:]:
  result = detect_ts(df[['측정일자',i]], max_anoms = 0.30, direction = 'both', threshold = 'med_max',alpha = 0.05)
  a = result['anoms'].index

  # 그것을 측정일자가 같은 데이터의 컬럼마다 nan값을 만들어 줬다.
  for j in a: 
    df.loc[df['측정일자'] == j, i] = np.nan 

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


- pyculiarity 방법 설명

# S-H-ESD 방법을 사용합니다.
T(trend) S(seasonal) C(cycle)을 통해서 각 값들을 추출후 median abusolute deviation(MAD)를 통해서 이상치를 탐지한다.

In [ ]:
# 결측치 확인하기
df.isnull().sum()

측정일자     0
동대문구    87
종로구     48
중구      58
성동구     46
금천구     62
송파구     51
관악구     64
강동구     47
서대문구    56
노원구     58
마포구     59
동작구     73
서초구     51
구로구     57
영등포구    52
강남구     58
강서구     76
성북구     50
중랑구     58
강북구     63
양천구     56
도봉구     53
은평구     57
용산구     57
광진구     50
dtype: int64

In [ ]:
# 결측치를 interpolate로 채워줬다.
for i in df.columns[1:]:
  df[i] = df[i].interpolate()

In [ ]:
df.isnull().sum()

측정일자    0
동대문구    0
종로구     0
중구      0
성동구     0
금천구     0
송파구     0
관악구     0
강동구     0
서대문구    0
노원구     0
마포구     0
동작구     0
서초구     0
구로구     0
영등포구    0
강남구     0
강서구     0
성북구     0
중랑구     0
강북구     0
양천구     0
도봉구     0
은평구     0
용산구     0
광진구     0
dtype: int64

In [ ]:
# 데이터의 저장을 한다.
df.to_csv('/content/drive/MyDrive/Proj_WT/DataSets/airpolution/pyculiarity_0.3.csv', index = False, encoding = 'utf-8')